## EDA

In [14]:
from __future__ import division
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, roc_curve
from sklearn import cross_validation
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from pandas.tools.plotting import scatter_matrix
import matplotlib.pyplot as plt
%matplotlib inline

import re
from HTMLParser import HTMLParser
import datetime
import cPickle as pickle

In [ ]:
# Files that contain extracted names and summaries
sample10_file = '/Users/lekha/galvanize/capstone/projectRiley/data/sample10/out.txt'
sample1000_file = '/Users/lekha/galvanize/capstone/projectRiley/data/out1000.txt'
all_file = '/Users/lekha/galvanize/capstone/projectRiley/data/outall.txt'

In [15]:
# Files that contain extracted names, summaries, title, industry, location and html
holyoke_raw = '/Users/lekha/galvanize/capstone/projectRiley/data/withfirstname/withfirstname_holyoke.txt'
utaustin_raw = '/Users/lekha/galvanize/capstone/projectRiley/data/withfirstname/withfirstname_utaustin.txt'
vassar_raw = '/Users/lekha/galvanize/capstone/projectRiley/data/withfirstname/withfirstname_vassar.txt'
wellesley_raw = '/Users/lekha/galvanize/capstone/projectRiley/data/withfirstname/withfirstname_wellesley.txt'
caltech_raw = '/Users/lekha/galvanize/capstone/projectRiley/data/withfirstname/withfirstname_caltech.txt'
gatech_raw = '/Users/lekha/galvanize/capstone/projectRiley/data/withfirstname/withfirstname_gatech.txt'
huskies_raw = '/Users/lekha/galvanize/capstone/projectRiley/data/withfirstname/withfirstname_huskies.txt'

# Create one DF out of the bunch

In [ ]:
# Files that contain extracted names, summaries, title, industry, location and html
files = ['/Users/lekha/galvanize/capstone/projectRiley/data/withfirstname/withfirstname_holyoke.txt', 
'/Users/lekha/galvanize/capstone/projectRiley/data/withfirstname/withfirstname_utaustin.txt',
'/Users/lekha/galvanize/capstone/projectRiley/data/withfirstname/withfirstname_vassar.txt',
'/Users/lekha/galvanize/capstone/projectRiley/data/withfirstname/withfirstname_wellesley.txt',
'/Users/lekha/galvanize/capstone/projectRiley/data/withfirstname/withfirstname_caltech.txt',
'/Users/lekha/galvanize/capstone/projectRiley/data/withfirstname/withfirstname_gatech.txt',
 '/Users/lekha/galvanize/capstone/projectRiley/data/withfirstname/withfirstname_huskies.txt']

outfile = '/Users/lekha/galvanize/capstone/projectRiley/data/withfirstname/withfirstname_all.txt'

def combine_dfs(infile_list, out_file):
    cnt = 1
    for f in infile_list:
        if cnt == 1:
            df = pd.read_csv(f, sep="|")
        else:
            df = pd.concat([df, pd.read_csv(f, sep="|")], axis=0)
        cnt += 1
    print ("Concatenated {0} files into a single data frame").format(cnt)
    
    df.to_csv(out_file, sep="|")
    return df
 

df = combine_dfs(files, outfile)

In [ ]:
df = pd.concat([df_uwash, df_holyoke, df_utaustin, df_gatech, df_wellesley, df_vassar, df_caltech], axis=0)

## Gender Prediction using First Names

In [34]:
df = pd.read_csv(caltech_raw, sep="|")

In [17]:
def gender_features(word):
    '''
    Returns the following features given a word:
    - name
    - last letter
    - last 2 letters
    - boolean if last letter is vowel
    '''
    return {'name': word, 'last_letter': word[-1], 'last_2_letters': word[-2:], 'last_is_vowel': (word[-1] in 'aeiou')}

In [35]:
import nltk
from nltk.corpus import names

def gender_classifier():
    # 7944 labeled names from nltk corpus
    labeled_names = ([(name, 'male') for name in names.words('male.txt')] +[(name, 'female') for name in names.words('female.txt')])

    # nltk's NaiveBayesClassifier takes featuresets to train that is a tuple of (features_dict, label)
    featuresets_train = [(gender_features(name), gender) for (name, gender) in labeled_names]
    classifier = nltk.NaiveBayesClassifier.train(featuresets_train)

    return classifier

def gender_predict(first_name, classifier):
    # get the features of the set to be classified
    name_features = gender_features(first_name)
    g = classifier.classify(name_features)

    return g

def force_gender(name, gender, males, females):
    if name in females:
        return 'female'
    if name in males:
        return 'male'
    return gender

def clean_name(first_name):
    if first_name == 'nan':
        return 'realnan'
    return first_name


In [36]:
df['first_name'] = df['first_name'].apply(lambda x:clean_name(x))

In [37]:
df['first_name'].fillna(value='realnan', inplace=True)

In [38]:
my_fem = pd.read_csv('/Users/lekha/galvanize/capstone/projectRiley/data/genderpredict/my_female.txt', header=None)
my_male = pd.read_csv('/Users/lekha/galvanize/capstone/projectRiley/data/genderpredict/my_male.txt', header=None)
females = list(np.array(my_fem).flatten())
males = list(np.array(my_male).flatten())

classifier = gender_classifier()

In [39]:
df['gender'] = df['first_name'].apply(lambda x: gender_predict(x, classifier))

In [10]:
df['gender_forced'] = df.apply(lambda x: force_gender(x['first_name'], x['gender'], males, females), axis=1)

In [ ]:
# Adding names to my labeled_name list

male_names = [name.lower() for name in names.words('male.txt')]
female_names = [name.lower() for name in names.words('female.txt')]

my_fem = pd.read_csv('/Users/lekha/galvanize/capstone/projectRiley/data/genderpredict/my_female.txt', header=None)
my_male = pd.read_csv('/Users/lekha/galvanize/capstone/projectRiley/data/genderpredict/my_male.txt', header=None)

my_females = list(np.array(my_fem).flatten())
my_males = list(np.array(my_male).flatten())

male_names = set(male_names + my_males)
female_names = set(female_names + my_females)
    
len(male_names)
len(female_names)

In [ ]:
len(female_names)

In [ ]:
# Verification Cell for Classification Algorithm
train_set, test_set = featuresets[600:], featuresets[:500]
print classifier.classify(gender_features('rahul'))
print classifier.classify(gender_features('Trinity'))
print(nltk.classify.accuracy(classifier, test_set))
classifier.show_most_informative_features(5)

In [ ]:
df['first_name'].nunique()

In [ ]:
df_agg = df.groupby(['gender','first_name']).agg({'counter':sum})

In [ ]:
#group by the first level of the index
g = df_agg['counter'].groupby(level=0, group_keys=False)

In [ ]:
# sort by the count of the first names
g.nlargest(50)

In [ ]:
females = df[df['gender'] == 'female']
males = df[df['gender'] == 'male']

In [ ]:
females['first_name'].nunique()

In [ ]:
males['first_name'].nunique()

In [ ]:
set(males['first_name'])

In [ ]:
# unique names in females and males
print (females_all['first_name'].nunique())
print (males_all['first_name'].nunique())

In [ ]:
x = zip(fnames, gender_fnames1)
with open('gender1000.txt', 'w') as f:
    for line in x:
        name = line[0]
        g = line[1]
        nameg = name + "," + g + "\n"
        f.write(nameg)
        #f.write("\n")

In [ ]:
letter_freq = df.last_letter.value_counts()
letter_freq.sort_values(ascending=False, inplace=True)


In [ ]:
# Plot one of the features for gender prediction: last letter
fig, ax = plt.subplots()
letter_freq.plot(ax=ax, kind='bar', alpha = 0.5, figsize=(12,8))